In [1]:
from collections import *
import random as rnd
import os
import math as m

In [2]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
!mv shakespeare_input.txt generation/shakespeare_input.txt

--2017-07-05 11:01:56--  http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573338 (4.4M) [text/plain]
Saving to: ‘shakespeare_input.txt’

shakespeare_input.t 100%[===================>]   4.36M  1.09MB/s    in 4.9s    

2017-07-05 11:02:01 (913 KB/s) - ‘shakespeare_input.txt’ saved [4573338/4573338]



In [2]:
fname = 'generation/shakespeare_input.txt'
with open(fname, 'r') as f:
    data = f.read().lower() 

In [17]:
def train_ngram_char_lm(datas, n=5):
    order = n-1
    lm = defaultdict(Counter)
    pad = "~" * order
    for data in datas:
        data = pad + data + pad

        for i in range(len(data) - order):
            history, char = data[i:i+order], data[i+order]
            lm[history][char]+=1
        
    def normalize(counter):
        s = float(sum(counter.values()))
        return {c:cnt/s for c,cnt in counter.items()}
    
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

def train_unigram_char_lm(datas):
    data = '\n'.join(datas)
    return { c:data.count(c)/len(data) for c in set(data) }

In [18]:
unigram_model = train_unigram_char_lm([data])

In [19]:
five_model = train_ngram_char_lm([data], n=5)

In [30]:
def generate_random(apriori_char_probs):
    random_point = rnd.random()
    s = 0
    for char, proba in apriori_char_probs.items():
        s += proba
        if s > random_point: return char

def generate(model, n, max_len=1000, seed = None, unigram_model = None):
    rnd.seed(seed)
    prev = '~'*(n-1)
    result = ''
    i = 0
    while(True):
        i += 1
        if n == 1:
            apriori_char_probs = model
        else:
            apriori_char_probs = model[prev]
            
        char = generate_random(apriori_char_probs)
        if(char == "~" or i > max_len): break
            
        result += char
        prev = prev[1:] + char
        
    return result

In [31]:
print(generate(five_model, 5, max_len=100))

first loved,
saving in his for should to
me on.

claudio:
look you speaking! i hath the quiet, oft f


In [32]:
# let's try Stus poetry
def readfile(fname):
    with open(fname, 'r') as f:
        return f.read().lower()
    
def preprocess(poem):
    return poem.replace('\t', ' ')
    

folder = 'generation/stus/'
stus_poetry = [preprocess(poem) for fname in os.listdir(folder) for poem in readfile(folder+fname).split('|')]
all_text = '\n'.join(stus_poetry)

In [45]:
stus_big_model = train_ngram_char_lm(stus_poetry, n=10)
stus_unigram_model = train_unigram_char_lm(stus_poetry)

In [48]:
poem = generate(stus_big_model, 10, unigram_model=stus_unigram_model)
print(poem)

﻿ несуть в руках, пропахлі небуттям.
світ твориться і міниться, мов око
побожеволілі, знечулі
чотири мури,
три двері, ґрачене вікно,
в якому районі міста
пленуми проводять нас поранні дзвони від грузькому
скотилися повільно побивала. слава богу,
пізнавши вочевидь: любов,
як і життя — не одійду за сталою
імлою, за збіглим сном під чорною ганьбою,
межи слідами чорних спроб, і кожна хвиля
у берегах, мов голуба, пустила в лет
там, де чотири вітри,
ласкаві ластівки на електричних дротах,
почорнілий пень підкаже,
де знайти загублена в міжпланетарній пустці —
в’язниця, що донесла з говерли
тривожні породіль. нема одвіку
легкої віри і легких утіх.


In [42]:
def plagiarism_check(text, big_text, window_size):
    for window_start in range(len(text)-window_size):
        window_text = text[window_start:window_start + window_size]
        index = big_text.find(window_text)
        
        if(index > -1):
            s = max(index, 0)
            e = min(index + window_size, len(big_text))
            print("Plagiarism found: {}".format(big_text[s:e]))
            return True
    return False

plagiarism_check(poem, all_text, 50)

False